In [3]:
"""
Boiler plate imports
"""
#  Standard Imports
import matplotlib.pyplot as plt
plt.rc('font', family='serif', size=14)
import pandas as pd
import numpy as np
import seaborn as sns

#  Helpers
import os
import re

#  Cheminformatics
import rdkit



In [11]:
#  Helper functions

def distance(a, b):
    """
    Distance using the norm
    """
    a = [x[1:] for x in a]
    b = [x[1:] for x in b]

    return np.linalg.norm(np.array(a) - np.array(b))

def difference(a, b):
    a = [x[1:] for x in a]
    b = [x[1:] for x in b]
    return np.array(a) - np.array(b)


def file_name_to_code(filename):
    return int(re.sub('[^0-9]','', filename))

def g09_input_to_geometry(g09_input):
    """
    Takes a list of lines in a com file and returns a geometry in the form:
    [[str(atom name), float(x), float(y), float(z)], ...]
    """
    geom = []
    for x in g09_input:
        spl = x.split()
        if len(spl) == 4 and spl[0] in atom_names:
            geom.append([spl[0], float(spl[1]), float(spl[2]), float(spl[3])])
    return geom

def pun_to_dipole_moment(pun_file):
    """
    Takes a list of lines from a pun file and returns the dipole moment in the form:
    [[str(atom name), float(x), float(y), float(z)], ...]
    """
    dipoles = []
    for x in pun_file:
        if x.__contains__("Rank "):
            spl = x.split()
            if len(spl) == 6 and spl[0] in atom_names:
                dipoles.append([spl[0], float(spl[1]), float(spl[2]), float(spl[3])])
    return dipoles

def pun_to_charge(pun_file):
    """
    Takes a list of lines from a pun file and returns the charge in the form:
    [[str(atom name), float(x)]
    """
    charges = []
    for i, x in enumerate(pun_file):
        if x.__contains__("Rank "):
            spl = x.split()
            if len(spl) == 6 and spl[0] in atom_names:
                charges.append([spl[0], float(pun_file[i+1])])
    return charges

def pun_to_charge(pun_file):
    """
    Takes a list of lines from a pun file and returns the charge in the form:
    [[str(atom name), float(x)]
    """
    charges = []
    for i, x in enumerate(pun_file):
        if x.__contains__("Rank "):
            spl = x.split()
            if len(spl) == 6 and spl[0] in atom_names:
                charges.append([spl[0], float(pun_file[i+1])])
    return charges

def pun_to_R1(pun_file):
    """
    Takes a list of lines from a pun file and returns the charge in the form:
    [[str(atom name), float(x)]
    """
    charges = []
    for i, x in enumerate(pun_file):
        if x.__contains__("Rank "):
            spl = x.split()
            if len(spl) == 6 and spl[0] in atom_names:
                r = pun_file[i+2].split()
                r = [float(x) for x in r]
                charges.append([spl[0], *r])
    return charges

def pun_to_R2(pun_file):
    """
    Takes a list of lines from a pun file and returns the charge in the form:
    [[str(atom name), float(x)]
    """
    charges = []
    for i, x in enumerate(pun_file):
        if x.__contains__("Rank "):
            spl = x.split()
            if len(spl) == 6 and spl[0] in atom_names:
                r = pun_file[i+3].split()
                r = [float(x) for x in r]
                charges.append([spl[0], *r])
    return charges

def dihedral(p):
    """
    Copied from stack overflow: 
    https://stackoverflow.com/questions/20305272/
    dihedral-torsion-angle-from-four-points-in-cartesian-coordinates-in-python

    Praxeolitic formula
    1 sqrt, 1 cross product"""
    
    p0 = p[0]
    p1 = p[1]
    p2 = p[2]
    p3 = p[3]

    b0 = -1.0*(p1 - p0)
    b1 = p2 - p1
    b2 = p3 - p2

    # normalize b1 so that it does not influence magnitude of vector
    # rejections that come next
    b1 /= np.linalg.norm(b1)

    # vector rejections
    # v = projection of b0 onto plane perpendicular to b1
    #   = b0 minus component that aligns with b1
    # w = projection of b2 onto plane perpendicular to b1
    #   = b2 minus component that aligns with b1
    v = b0 - np.dot(b0, b1)*b1
    w = b2 - np.dot(b2, b1)*b1

    # angle between v and w in a plane is the torsion angle
    # v and w may not be normalized but that's fine since tan is y/x
    x = np.dot(v, w)
    y = np.dot(np.cross(b1, v), w)
    return np.degrees(np.arctan2(y, x))

            

In [25]:
systems = ["acrolein", "butane", "butadiene", "water", "methanol", "cyanomethane"]
systems = ["acrolein", "butane", "butadiene", "water", "methanol", "cyanomethane"]

for system in systems:    
    
    atom_names = ["C", "O", "H", "N"]

    acrolein_geom_path = "/home/boittier/Documents/MDCM/scripts/{}_opt.xyz".format(system)
    acrolein_pun_path = "/home/boittier/Documents/MDCM/big_basis_set/GDMA/{}_opt.pun".format(system)

    acrolein_ref_geom = g09_input_to_geometry(open(acrolein_geom_path, "r").readlines())
    acrolein_ref_pun = pun_to_dipole_moment(open(acrolein_pun_path).readlines())
    acrolein_ref_charge = pun_to_charge(open(acrolein_pun_path).readlines())
    acrolein_ref_R1 = pun_to_R1(open(acrolein_pun_path).readlines())
    acrolein_ref_R2 = pun_to_R2(open(acrolein_pun_path).readlines())

    #  Fixing an inconsistency :( 
    if system == "water":
        system = "h2o"
    
    path_to_g09_input = "/home/boittier/Documents/MDCM/nms_sampling/{}_geom.xyz.dir/energies".format(system)
    path_to_GDMA_output = "/home/boittier/Documents/MDCM/GDMA_output/{}".format(system)

    com_files = [ x for x in os.listdir(path_to_g09_input) if x.__contains__("com")]
    pun_files = [ x for x in os.listdir(path_to_GDMA_output) if x.__contains__("pun")]


    number_to_geometry = {}
    for com_file in com_files:
        number_to_geometry[file_name_to_code(com_file) ] = g09_input_to_geometry(
            open(os.path.join(path_to_g09_input, com_file)).readlines())

    number_to_punch = {}
    for pun_file in pun_files:
        f = open(os.path.join(path_to_GDMA_output, pun_file)).readlines()
        if len(pun_to_dipole_moment(f)) < 1:
            #print("failed? ", pun_file)
            pass
        else:
            number_to_punch[file_name_to_code(pun_file)] = pun_to_dipole_moment(f)

    number_to_charge = {}
    for pun_file in pun_files:
        f = open(os.path.join(path_to_GDMA_output, pun_file)).readlines()
        if len(pun_to_charge(f)) < 1:
            #print("failed? ", pun_file)
            pass
        else:
            number_to_charge[file_name_to_code(pun_file)] = pun_to_charge(f)
            
            
    number_to_R1 = {}
    for pun_file in pun_files:
        f = open(os.path.join(path_to_GDMA_output, pun_file)).readlines()
        if len(pun_to_charge(f)) < 1:
            #print("failed? ", pun_file)
            pass
        else:
            number_to_R1[file_name_to_code(pun_file)] = pun_to_R1(f)
            
            
    number_to_R2 = {}
    for pun_file in pun_files:
        f = open(os.path.join(path_to_GDMA_output, pun_file)).readlines()
        if len(pun_to_charge(f)) < 1:
            #print("failed? ", pun_file)
            pass
        else:
            number_to_R2[file_name_to_code(pun_file)] = pun_to_R2(f)

    keys = list(number_to_punch.keys())

    rmsds = []
    delta_dipoles = []
    dihedrals = []
    charges = []
    delta_R1 = []
    delta_R2 = []

    for x in keys:
        rmsds.append(distance(acrolein_ref_geom, number_to_geometry[x]))
        
        delta_dipoles.append(distance(acrolein_ref_pun, number_to_punch[x]))
        
        charges.append(distance(acrolein_ref_charge, number_to_charge[x]))
        delta_R1.append(distance(acrolein_ref_R1, number_to_R1[x]))
        delta_R2.append(distance(acrolein_ref_R2, number_to_R2[x]))
        
        geom = [x[1:] for x in number_to_geometry[x]] 
        #dih_atoms = np.array([geom[3], geom[2], geom[0], geom[1]])
        #dihedrals.append(dihedral(dih_atoms)) ## should remove bc water

    #d = {"$\Delta$dipole": delta_dipoles, "Dihedral": dihedrals, "RMSD": rmsds, "$\Delta$R0": charges}
    d = {"$\Delta$dipole": delta_dipoles, "RMSD": rmsds,
         "$\Delta$R0": charges, "$\Delta$R1": delta_R1, "$\Delta$R2": delta_R2}
    df = pd.DataFrame(d)   
    
    print(system, " ", max(charges))
    
    g = sns.pairplot(df);
    g.map_offdiag(sns.kdeplot, n_levels=6);
    plt.savefig("{}_pairplot.pdf".format(system))
    plt.clf()

    g = sns.jointplot("RMSD", "$\Delta$R0", data=df,
                      kind="reg", truncate=False, height=7)
    plt.savefig("{}_rmsd_vs_charge.pdf".format(system))
    plt.clf()
    
    g = sns.jointplot("RMSD", "$\Delta$R1", data=df,
                      kind="reg", truncate=False, height=7)
    plt.savefig("{}_rmsd_vs_R1.pdf".format(system))
    plt.clf()
    
    g = sns.jointplot("RMSD", "$\Delta$R2", data=df,
                      kind="reg", truncate=False, height=7)
    plt.savefig("{}_rmsd_vs_R2.pdf".format(system))
    plt.clf()
    
    d = {"key": keys, "$\Delta$dipole": delta_dipoles, "RMSD": rmsds,
         "$\Delta$R0": charges, "$\Delta$R1": delta_R1, "$\Delta$R2": delta_R2}
    df = pd.DataFrame(d) 
    df = df.sort_values("$\Delta$R0", ascending=False)
    df.to_csv("{}_data.csv".format(system))



acrolein   0.0847975886649271
butane   0.46565998264893743
butadiene   0.04469487591412789
h2o   0.10769586542894645
methanol   0.3543278372280695
cyanomethane   0.07025391239688714


/home/boittier/miniconda3/lib/python3.8/site-packages/seaborn/axisgrid.py:1287: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, axes = plt.subplots(len(y_vars), len(x_vars),


<Figure size 900x900 with 0 Axes>

<Figure size 504x504 with 0 Axes>

<Figure size 504x504 with 0 Axes>

...

<Figure size 504x504 with 0 Axes>

<Figure size 900x900 with 0 Axes>

<Figure size 504x504 with 0 Axes>

<Figure size 504x504 with 0 Axes>

<Figure size 504x504 with 0 Axes>

,key,$\Delta$dipole,RMSD,$\Delta$R0,$\Delta$R1,$\Delta$R2
669,1252,6.967082,0.166594,0.070254,1.135334,0.483453
1,1785,7.082464,0.156541,0.065472,1.122026,0.474059
747,1432,6.933635,0.132065,0.063610,1.118006,0.447314
248,1492,6.956245,0.118932,0.063323,1.123029,0.429369
737,1688,7.084762,0.158674,0.062876,1.146553,0.444834
...,...,...,...,...,...,...
530,174,2.364951,0.091940,0.004971,0.221136,0.561979
664,13,2.414421,0.062047,0.004714,0.225684,0.557965
304,160,7.027584,0.069839,0.004604,1.128553,0.425582
498,1780,0.474850,0.145010,0.004238,0.047604,0.236581
